# Logstash

- 실제 들어오는 데이터를 변형하거나 추가하는 기능.
- 다양한 input, output pipeline 제공.
- 수집된 데이터는 elastic search로 들어감.
- hadoop eco system 데이터 수집 가능.

- 데이터 흐름을 위한 오픈소스 중앙 처리 엔진.(Beats와 함께 사용 권장)
- dataflow 파이프라인을 구축하여 이벤트 데이터 변환 및 스트림 설정.
    * 작게 시작하여 파이프라인을 늘려가는 방식 권장.
    * 작은 데이터를 색인해보며 확장.
- 다양한 데이터 자원 접근.
- 원시 버퍼링을 통한 수평적 스케일링
- 통합 처리를 위한 탄탄한 플러그인 환경.

### Pipeline 강화
- 데이터스트림 실시간 변형, 제거, 정규화
- 조건을 활용한 데이터 흐름 분리
- 파이프라인 활성 상태 모니터링
- SSL/TLS를 활용한 파이프라인 보안

___

동작 순서
1. input: 데이터 수집 source에 대한 정의.
    - 파일, syslog, sql query, http request, elastic search, beats, ...
2. filter: 들어온 데이터를 어떻게 가공할지에 대한 정의.
    - log 파싱, 데이터 추가, 태그 추가, ...
3. output: 데이터를 어디로 보낼 것인지에 대한 정의.
    - elastic search, 데이터 보관소, monitoring system, ...

## Pipeline: 데이터 처리를 위한 Logstash 설정
- Codec: 데이터 인코딩, 디코딩을 위한 명세.
    * json, avro, msgpack, netflow, cloudtrail, text, ...
    * 원하는 모양으로 가공하여 전송 가능.
- Work queue: Logstash의 filter가 작동하는 부분.
    * worker를 늘려 성능 향상 가능.
- Batcher: elastic search에 데이터를 보내기 위해 기본적으로 batch를 만들어 전송.
    * 항상 Bulk api를 사용해야 좋은 성능.
    * batch => bulk api로 전송하는 순서.

___

logstash.yml
- node.name: node 이름 지정 가능.
- pipeline.workers: 더 많은 cpu core를 사용하고자 하면 늘리면 된다.
- pipeline.output.workers: output별로 작업을 담당할 worker 수 지정 가능.
- pipeline.batch.size: 들어오는 event data(log 한 라인)를 몇개씩 묶어 batch를 만들 것인지 지정.
- pipeline.batch.delay: 몇초마다 원하는 목적지로 데이터를 보낼 것인지 지정.
 
 => batch.size는 늘리고 batch.delay는 줄이면 보다 많은 작업 실행 가능(heap memory 설정도 병행해야함)
 
- Logstash: 실행 시 configuration 설정해준 다음 실행해줘야 함.(input, output 정의)
- config.reload.automatic: true로 설정시 config 파일 내용 변경을 자동으로 반영.
- config.reload.delay: 몇초마다 변경을 감지할 것인지.
- path.config: 어떤 경로에서 config 파일을 가져올것인지 지정.

pipeline.yml
- pipeline을 어디서 읽어올지 지정 가능.

___

### Logstash 실행
- bin/logstash -e 'input { stdin { } } output { stdout { } }': input = standard input, output = standard output으로 설정하여 실행.
- bin/logstash -f simple.config: 미리 생성한 simple.config의 설정 내용을 반영하여 Logstash 실행.
    * rubydebug 형식: version, host, timestamp를 자동으로 입력, 입력받은 모든 데이터는 message field 내에 입력됨.
    * input codec 따로 지정 안하면 입력한 내용 그대로 들어감.(codec 지정시 지정된 codec 형식에 맞춰 들어감)
    

    input{
     tcp{
      port => 9900
     }
    }

    filter{
     grok{
      match => { "message" => "Hello %{WORD:name}" }
     }
    }

    output{
     stdout{
      codec => rubydebug
     }
    }
- 입력: tcp로 받음
- grok filter: message 내용을 name이라는 field에 저장.
- output: stdout


    입력: echo 'Hello Jaehyun' | nc localhost 990
    
    결과: 
    {
      "@version" => "1",
       "message" => "Hello Jaehyun",
          "name" => "Jaehyun",
          "port" => 61177,
    "@timestamp" => 2018-08-12T17:10:37.460Z,
          "host" => "localhost"
    }

    input{
     tcp{
      port => 9900
      type => "apache"
     }
    }

    filter{
     if [type] == "apache"{

      grok{
       match => {
        message => "%{COMBINEDAPACHELOG}"
        remove_field => "message"
       }
      }

      geoip{
       source => "clientip"
       fields => ["city_name", "country_name", "location", "region_name"]
      }

      date{
       match => ["timestamp", "dd/MMM/yyyy:HH:mm:ss Z"]
       remove_field => "timestamp"
      }
     }
    }

    output{
     elasticsearch{
      hosts => ["localhost:9200"]
     }
    }
- input: tcp에 apache type 추가
- filter
    * COMBINEDAPACHELOG: 로그 파싱을 위해 미리 정의되어 있음.
    * geoip: ip주소를 통해 사용자의 지도상 위치를 가져올 수 있음.
    * date: remove_field 사용 이유 - Logstash가 기본적으로 찍는 timestamp를 우리가 정의한 timestamp로 대체하기 위해 사용.
- logstash가 알아서 config 바뀐 것을 감지하고 재로딩.


    입력: 14.49.42.25 - - [10/Nov/2017:01:24:06 +0000] "GET /articles/dynamic-dns-with-dhcp/ HTTP/1.1" 200 18848 "-" "Mozilla/5.0 (Windows; U; Windows NT 6.1; en-US; rv:1.9.2b1) Gecko/20091014 Firefox/3.6b1 GTB5"
    결과: elastic search에 log 내용 입력됨.
    
    
curl localhost:9600/_node?pretty: logstash 정보 가져올 수 있음.

사용 사례
- twitter로 input 받을 수 있음. => twitter 내용을 elastic search로 실시간 색인 가능.
- beats로 input: 출력을 logstash로 하여 필터링 후 output으로 elastic search.
- output을 여러개 둘 수 있음: 동시에 output 전송 가능.
- jdbc plugin 사용 => sql에서 가져올 수 있음.
    * query를 통해 가져올 데이터 범위 지정 가능.